###### ### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2024 Semester 1

## Assignment 1: Wine quality classification with K-NN


**Student ID(s):**     `1172598`


This iPython notebook is a template which you will use for your Assignment 1 submission.

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

**Adding proper comments to your code is MANDATORY. **

## 1. K-NN classifier

In [85]:
import pandas as pd
import random

wine_train = pd.read_csv('winequality-train.csv')
wine_test = pd.read_csv('winequality-test.csv')

def KNN_Classifier(k, df_test, df_train):
    labels = []

    # Iterate through test data
    for index, row in df_test.drop(columns=['quality']).iterrows():
        distance_arr = []

        # Iterate through training data
        for index1, row1 in df_train.drop(columns=['quality']).iterrows():
            # Call function that calculates distance and add to distance array
            distance_arr.append(Calc_Distance(len(df_test.columns)-1, row, row1))

        # Add distance array to dataframe and sort it by distance
        df_temp = df_train.assign(distance=distance_arr)
        df_temp = df_temp.sort_values('distance')

        labels.append(Vote(k, df_temp))
    return labels

# Function to calculate distance of multi dimensional arrays
def Calc_Distance(length, row, row1):
    # Value used for distance i.e (a-b)^2
    dis = 0

    # Iterate through all attributes of both test and training instances
    for i in range(length):
        # Add each dis value for each attribute for instances from test and training
        dis = dis + pow(row.iloc[i] - row1.iloc[i], 2)
    
    # Return multi dimensional distance
    return pow(dis, 1/2)

# Conditionals to check voting criteria
def Vote(k, df_temp):
    if (k == 1):
        label = df_temp['quality'].iloc[0]
    else:
        # Nearest quality predictions, note the distance is already sorted
        values = df_temp[['quality', 'distance']][0:k].reset_index()

        # Iterate through k values of predicted quality and count good and bad predictions
        good = 0
        bad = 0
        for index, row in values.iterrows():
            if (row['quality'] == 0):
                bad = bad + 1
            else:
                good = good + 1
        # Check each outcome
        if (good == bad):
            # This is case where 
            dups = values['distance'].duplicated().tolist()
            # Check if duplicates exits
            if True in dups:
                count = 0
                # Get all duplicates of nearest distance, ignore other duplicates
                for index in range(len(dups)):
                    if (dups[index] == True):
                        if (count == 0):
                            start = index - 1
                        count = count + 1
                        if (index == 9):
                            end = start + count
                            break
                    elif ((dups[index] == False and count != 0) or (dups[index] == True and index == 9)):
                        end = start + count
                        break
                # All nearest duplicates
                same_dis = values['quality'][start:end].tolist()

                # If duplicates all have same quality just pick 1-NN
                if (len(set(same_dis)) == 1):
                    label = values['quality'].iloc[0]
                # Otherwise pick randomly amongst duplicates
                else:
                    label = random.choice(same_dis)
                    
            # Simply pick 1-NN if no duplicates
            else:
                label = values['quality'].iloc[0]
        elif (good > bad):
            label = 1
        else:
            label = 0
    return label
    

labels = KNN_Classifier(10, wine_test[0:50], wine_train)



## 2. 1-NN classification

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer must be submitted separately as a PDF.

In [89]:
def Find_Proportion(df_test, labels_classified):
    labels_test = df_test['quality'].tolist()
    count = 0
    for i in range(len(labels_classified)):
        if (labels_test[i] == labels_classified[i]):
            count = count + 1
    return float(count)/float(len(labels_classified))

print("{:0.4f}".format(Find_Proportion(wine_test, labels)))

0.7400


## 3. Normalization

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer must be submitted separately as a PDF.

## 4. Model extensions

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer must be submitted separately as a PDF.

### 4.1
Compare the performance of your best 1-NN model from Question 3 to a Gaussian naive Bayes model on this dataset (you may use library functions to implement the Gaussian naive Bayes model). In your write-up, state the accuracy of the naive Bayes model and identify instances where the two models disagree. Why do the two models classify these instances differently?

### 4.2
Implement two additional distance measures for your K-NN model: cosine similarity and Mahalanobis distance (you may use library functions for these distance measures). Do 1-NN classification using each of these new distance measures and the three normalization options from Question 3. Discuss how the new distance metrics compare to Euclidean distance and how each metric is affected by normalization.

### 4.3
Implement either of the two K-NN weighting strategies discussed in lecture (inverse linear distance or inverse distance). Compare the performance of the weighted and majority vote models for a few different values of K. In your write-up, discuss how weighting strategy and the value of K affect the model's decisions.

### 4.4
Measure the empirical distribution of class labels in the training dataset (what percentage of the training data comes from each class). Then evaluate the distribution of labels predicted by your K-NN model for the test data, for a range of values for K. Does the class distribution of the predicted labels match the class distribution of the training data? Explain why or why not.